In [ ]:
import pandas as pd
import numpy as np
import scipy

In [ ]:
train = pd.read_csv('Dataset/train.csv');

In [ ]:
train.head(1)

### 1. clean all the nulls in department

In [ ]:
def tree_clean_null(df):
    result = df.dropna(subset = ["DepartmentDescription"], how = "all")
    print("After cleaning null on department, the shape is ",result.shape)
    return result

In [ ]:
def other_clean_null(df):
    result = df.dropna(subset = ["Upc"], how = "all")
    print("After cleaning null on upc, the shape is ",result.shape)
    return result

In [ ]:
testing = train

In [ ]:
testing = other_clean_null(testing)

- Now we have (645693, 7)

### 2. set the positive count

In [ ]:
def set_two_counts(df):
    df["BuyCount"] = df["ScanCount"]
    df["BuyCount"][df['ScanCount'] < 0] = 0
    df["ReturnCount"] =  - df["ScanCount"]
    df["ReturnCount"][df['ScanCount'] > 0] = 0
    print("After set_two_counts, the shape is ", df.shape)
    return df

In [ ]:
testing = set_two_counts(testing)

### 3. remove similar terms in department

- clean "mens wear" to "menswear"

In [ ]:
def remove_similar_in_department(df):
    df.DepartmentDescription = df.DepartmentDescription.replace(["MENS WEAR"], "MENSWEAR")
    return df

In [ ]:
testing = remove_similar_in_department(testing)

In [ ]:
testing.DepartmentDescription.unique().shape

### 4. group by visitnumber

Count types of Upc products, Fineline products, and Departments per visit

- compute the distince types of a column per visit (column = upc, fl, dp)

In [ ]:
def types_per_visit(df, _type):
    newtype = _type + "TypesPerVisit"
    type_mat = df.groupby(['VisitNumber'])[_type].nunique().reset_index()
    type_mat = type_mat.rename(columns={_type: newtype})
    return type_mat

- merge on visitnumber

In [ ]:
def merge_types_per_visit(df,type_mat):
    df = df.merge(type_mat, 
                left_on='VisitNumber', 
                right_on='VisitNumber', 
                how='left')
    print("After merge_types_per_visit, the shape is ", df.shape)
    return df
    

- compute the count of items per (visit, column)
- (column = upc, fl, dp)

In [ ]:
def buy_types_per_visit(df, _type):
    newtype = "BuyCountPer" + _type +"Visit"
    type_mat = df.groupby(['VisitNumber', _type])["BuyCount"].aggregate(np.sum).reset_index()
    type_mat = type_mat.rename(columns={"BuyCount": newtype})
    return type_mat

- merge on (visit, column)

In [ ]:
def merge_buy_types_per_visit(df,type_mat,_type):
    df = df.merge(type_mat, 
                left_on=['VisitNumber', _type], 
                right_on=['VisitNumber', _type], 
                how='left')
    print("After merge_buy_types_per_visit, the shape is ", df.shape)
    return df

In [ ]:
upc_mat = types_per_visit(testing, "Upc")
fl_mat = types_per_visit(testing, "FinelineNumber")
dp_mat = types_per_visit(testing, "DepartmentDescription")

In [ ]:
testing = merge_types_per_visit(testing, upc_mat)
testing = merge_types_per_visit(testing, fl_mat)
testing = merge_types_per_visit(testing, dp_mat)

In [ ]:
fl_mat = buy_types_per_visit(testing, "FinelineNumber")
dp_mat = buy_types_per_visit(testing, "DepartmentDescription")

In [ ]:
testing = merge_buy_types_per_visit(testing, fl_mat, "FinelineNumber")
testing = merge_buy_types_per_visit(testing, dp_mat, "DepartmentDescription")

- sum bought products

In [ ]:
def sum_per_visit(df):
    newtype = "SumPerVisit"
    sum_mat = df.groupby('VisitNumber')["BuyCount"].aggregate(np.sum).reset_index()
    sum_mat = sum_mat.rename(columns={"BuyCount": newtype})
    return sum_mat

In [ ]:
def merge_sum_per_visit(df, sum_mat):
    df = df.merge(sum_mat, 
                left_on=['VisitNumber'], 
                right_on=['VisitNumber'], 
                how='left')
    print("After merge_sum_per_visit, the shape is ", df.shape)
    return df  

In [ ]:
sum_mat = sum_per_visit(testing)

In [ ]:
testing = merge_sum_per_visit(testing,sum_mat)

In [ ]:
testing.head()

### 5. transfer text into vector

- reshape department

In [ ]:
temp1 = pd.get_dummies(testing.DepartmentDescription).astype(int, copy=False)

- reshape weekday

In [ ]:
temp2 = pd.get_dummies(testing.Weekday).astype(int, copy=False)

In [ ]:
# Now we don't need these columns anymore
# testing.drop(['DepartmentDescription', 'Weekday'], inplace=True, axis=1)


# Concatenate df with the dummy dataframes after converting df to int
# testing.astype(int, copy=False)
testing = pd.concat([testing, temp1, temp2], axis=1)

In [ ]:
del temp1, temp2

In [ ]:
testing.shape

In [ ]:
testing.drop(['VisitNumber' ,'DepartmentDescription', 'Weekday'], inplace=True, axis=1)

In [ ]:
testing.shape

In [ ]:
testing.columns.values

In [ ]:
testing.index[testing.isnull().any(axis = 1)]

In [ ]:
testing.to_csv("Dataset/knn_data.csv")